# Preprocessing

#### Stage 1: Import relevant modules.

In [158]:
import pandas as pd
import sqlite3

#### Stage 2: Upload data files (.csv) into the environment as a dataframe.


In [159]:
# gets absolute path of current directory

# reads .csv file(s) data then stores it as a dataframe
dfCOA111 = pd.read_csv('cop504cwdata/22COA111ModuleRegister.csv')
dfCOA122 = pd.read_csv('cop504cwdata/22COA122ModuleRegister.csv')

In [160]:
dfCOA111

,sid_x,S1.W1\nMonday (03-10-2022)\n09:00 - 10:00\nLecture\nU020\n,S1.W1\nWednesday (05-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W2\nThursday (13-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W3\nWednesday (19-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W3\nWednesday (19-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W3\nThursday (20-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W4\nWednesday (26-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W4\nWednesday (26-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W4\nThursday (27-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W5\nWednesday (02-11-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W5\nWednesday (02-11-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W5\nThursday (03-11-2022)\n14:00 - 15:00\nLecture\nCC011\n
0,0,GPS,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,X,X,NaN,X
1,1,GPS,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
2,2,GPS,GPS,GPS,NaN,X,X,NaN,GPS,X,NaN,GPS,X,NaN,X
3,3,NaN,NaN,GPS,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS
4,4,GPS,X,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,238,X,X,GPS,NaN,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
232,239,GPS,X,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,X
233,240,GPS,GPS,GPS,NaN,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
234,241,GPS,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS


In [161]:
dfCOA122

,sid,S1.W1\nMonday (03-10-2022)\n14:00 - 16:00\nLecture\nCC012\n,S1.W1\nThursday (06-10-2022)\n09:00 - 11:00\nComputer Lab\nN001...\n,S1.W1\nThursday (06-10-2022)\n11:00 - 13:00\nComputer Lab\nN001...\n,S1.W2\nMonday (10-10-2022)\n14:00 - 16:00\nLecture\nCC012\n,S1.W2\nThursday (13-10-2022)\n09:00 - 11:00\nComputer Lab\nN001...\n,S1.W2\nThursday (13-10-2022)\n11:00 - 13:00\nComputer Lab\nN001...\n,S1.W3\nMonday (17-10-2022)\n14:00 - 16:00\nLecture\nCC012\n,S1.W3\nThursday (20-10-2022)\n09:00 - 11:00\nComputer Lab\nN001...\n,S1.W3\nThursday (20-10-2022)\n11:00 - 13:00\nComputer Lab\nN001...\n,S1.W4\nMonday (24-10-2022)\n14:00 - 16:00\nLecture\nCC012\n,S1.W4\nThursday (27-10-2022)\n09:00 - 11:00\nComputer Lab\nN001...\n,S1.W4\nThursday (27-10-2022)\n11:00 - 13:00\nComputer Lab\nN001...\n,S1.W4\nFriday (28-10-2022)\n12:00 - 13:00\nPersonal Best\nCC011\n,S1.W5\nMonday (31-10-2022)\n14:00 - 16:00\nLecture\nCC012\n,S1.W5\nTuesday (01-11-2022)\n10:00 - 11:00\nDrop in session\nWAV001\n[Excluded],S1.W5\nTuesday (01-11-2022)\n10:00 - 11:00\nDrop in session\nWAV001\n[Excluded].1,S1.W5\nThursday (03-11-2022)\n09:00 - 11:00\nTest\nN001...\n,S1.W5\nThursday (03-11-2022)\n11:00 - 13:00\nTest\nN001...\n,S1.W6\nMonday (07-11-2022)\n14:00 - 16:00\nLecture\nCC012\n
0,0,GPS,GPS,NaN,GPS,X,NaN,X,X,NaN,GPS,X,NaN,X,X,NaN,NaN,GPS,NaN,X
1,1,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,NaN,GPS,NaN,GPS
2,2,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,X,NaN,X,X,NaN,NaN,GPS,NaN,GPS
3,3,NaN,NaN,NaN,NaN,X,X,X,NaN,GPS,X,NaN,GPS,GPS,GPS,NaN,NaN,NaN,GPS,GPS
4,4,GPS,GPS,NaN,GPS,X,NaN,GPS,X,NaN,Ex,Ex,NaN,GPS,GPS,NaN,NaN,X,NaN,GPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,238,X,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,X,NaN,GPS,GPS,NaN,NaN,GPS,NaN,GPS
232,239,GPS,GPS,NaN,GPS,GPS,NaN,GPS,X,NaN,X,GPS,NaN,GPS,GPS,NaN,NaN,GPS,NaN,X
233,240,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,GPS,NaN,NaN,NaN,GPS,GPS
234,241,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,NaN,X,NaN,GPS


In [162]:
# creates copy to make module sessions data frame
dfCOA111Copy = dfCOA111.copy()
dfCOA122Copy = dfCOA122.copy()

# removes student ID for new 'dfCOA122Sessions' and 'dfCOA111Sessions' dataframe
dfCOA111Copy.drop('sid_x', axis = 1, inplace = True)
dfCOA122Copy.drop('sid', axis = 1, inplace = True) 

#### Stage 3: Create dfCOA122Sessions and dfCOA111Sessions dataframes.

In [163]:
############################# FUNCTIONS #############################

def splitColumnHeading(df, deliminator, initColumnName, newColumns):
    sessionInfo = df.columns.str.split(deliminator) # splits column headings by newline
    dfSessionInfo = pd.DataFrame(sessionInfo, columns=[initColumnName]) # stores the list of column information into a new data frame

    dfNew = dfSessionInfo[initColumnName].apply(pd.Series) 
    dfNew.columns = newColumns # creates new columns based on each value in the list 
    return dfNew

def splitColumnInfo(df, newColumns, columnToSplit, deliminator):
    df[newColumns] = df[columnToSplit].str.split(deliminator, expand = True)
    return df

def removeChars(df, columnName, deliminator):
    df[columnName] = df[columnName].str.strip(deliminator)
    return df

def formatForDatetime(df, date, times): # NOTE: 'times' list is in '%H:%M' format
    for time in times:
        df[time] = df[date] + ' ' + df[time].str.strip() + ':00'
    return df 

######################################################################

    3.a Store the details of each session (found in the column heading) into new dataframes.

In [164]:
newSessionColumns = ['Semester_Week', 'Date', 'Time', 'Lecture_Type', 'Room_ID', 'TO_DELETE']
initColumn = 'SessionInfo'
delim = '\n'

dfCOA111Sessions =  splitColumnHeading(dfCOA111Copy, delim, initColumn, newSessionColumns)
dfCOA122Sessions = splitColumnHeading(dfCOA122Copy,  delim, initColumn, newSessionColumns)

    3.b Split the column values into more distinct columns.

In [165]:
newColumns = [['Semester','Week'], ['Day','Date'], ['Start_Time', 'End_Time']]
oldColumns = ['Semester_Week', 'Date', 'Time']
delims = ['.', ' ', '-'] 

for col in range(len(oldColumns)):
    splitColumnInfo(dfCOA111Sessions, newColumns[col], oldColumns[col], delims[col])
    splitColumnInfo(dfCOA122Sessions, newColumns[col], oldColumns[col], delims[col])

    3.c Remove deliminators/redundant characters from column values.

In [166]:
columns = ['Date', 'Date', 'Semester', 'Week']
delims = ['(', ')', 'S', 'W']

for col in range(len(columns)):
    removeChars(dfCOA111Sessions, columns[col], delims[col])
    removeChars(dfCOA122Sessions, columns[col], delims[col])

# For dfCOA122Sessions '...' from the end of 'NOO1' must also be removed
removeChars(dfCOA122Sessions, 'Room_ID', '...')

,Semester_Week,Date,Time,Lecture_Type,Room_ID,TO_DELETE,Semester,Week,Day,Start_Time,End_Time
0,S1.W1,03-10-2022,14:00 - 16:00,Lecture,CC012,,1,1,Monday,14:00,16:00
1,S1.W1,06-10-2022,09:00 - 11:00,Computer Lab,N001,,1,1,Thursday,09:00,11:00
2,S1.W1,06-10-2022,11:00 - 13:00,Computer Lab,N001,,1,1,Thursday,11:00,13:00
3,S1.W2,10-10-2022,14:00 - 16:00,Lecture,CC012,,1,2,Monday,14:00,16:00
4,S1.W2,13-10-2022,09:00 - 11:00,Computer Lab,N001,,1,2,Thursday,09:00,11:00
5,S1.W2,13-10-2022,11:00 - 13:00,Computer Lab,N001,,1,2,Thursday,11:00,13:00
6,S1.W3,17-10-2022,14:00 - 16:00,Lecture,CC012,,1,3,Monday,14:00,16:00
7,S1.W3,20-10-2022,09:00 - 11:00,Computer Lab,N001,,1,3,Thursday,09:00,11:00
8,S1.W3,20-10-2022,11:00 - 13:00,Computer Lab,N001,,1,3,Thursday,11:00,13:00
9,S1.W4,24-10-2022,14:00 - 16:00,Lecture,CC012,,1,4,Monday,14:00,16:00


    3.d Reformat 'Start_Time' and 'End_Time' into datetime format.
    

In [167]:
oldFormatColumns = ['Start_Time', 'End_Time']
dateColumn = 'Date'

dfCOA111Sessions = formatForDatetime(dfCOA111Sessions, dateColumn, oldFormatColumns)
dfCOA122Sessions = formatForDatetime(dfCOA122Sessions, dateColumn, oldFormatColumns)

    3.e Add column name 'Session Number' and remove redundant columns.

In [168]:
dfCOA111Sessions['Session_Number'] = dfCOA111Sessions.index + 1 # Since session numbers do not start from 0
dfCOA122Sessions['Session_Number'] = dfCOA122Sessions.index + 1

toRemove = ['Semester_Week', 'Time', 'TO_DELETE']
dfCOA111Sessions.drop(toRemove, axis = 1, inplace = True) 
dfCOA122Sessions.drop(toRemove, axis = 1, inplace = True) 

    3.f Display final dfCOA111Sessions and dfCOA122Sessions

In [169]:
dfCOA111Sessions

,Date,Lecture_Type,Room_ID,Semester,Week,Day,Start_Time,End_Time,Session_Number
0,03-10-2022,Lecture,U020,1,1,Monday,03-10-2022 09:00:00,03-10-2022 10:00:00,1
1,05-10-2022,Lecture,SMB014,1,1,Wednesday,05-10-2022 10:00:00,05-10-2022 11:00:00,2
2,12-10-2022,Lecture,SMB014,1,2,Wednesday,12-10-2022 10:00:00,12-10-2022 11:00:00,3
3,12-10-2022,Tutorial,,1,2,Wednesday,12-10-2022 12:00:00,12-10-2022 13:00:00,4
4,13-10-2022,Lecture,CC011,1,2,Thursday,13-10-2022 14:00:00,13-10-2022 15:00:00,5
5,19-10-2022,Lecture,SMB014,1,3,Wednesday,19-10-2022 10:00:00,19-10-2022 11:00:00,6
6,19-10-2022,Tutorial,,1,3,Wednesday,19-10-2022 12:00:00,19-10-2022 13:00:00,7
7,20-10-2022,Lecture,CC011,1,3,Thursday,20-10-2022 14:00:00,20-10-2022 15:00:00,8
8,26-10-2022,Lecture,SMB014,1,4,Wednesday,26-10-2022 10:00:00,26-10-2022 11:00:00,9
9,26-10-2022,Tutorial,,1,4,Wednesday,26-10-2022 12:00:00,26-10-2022 13:00:00,10


In [170]:
dfCOA122Sessions

,Date,Lecture_Type,Room_ID,Semester,Week,Day,Start_Time,End_Time,Session_Number
0,03-10-2022,Lecture,CC012,1,1,Monday,03-10-2022 14:00:00,03-10-2022 16:00:00,1
1,06-10-2022,Computer Lab,N001,1,1,Thursday,06-10-2022 09:00:00,06-10-2022 11:00:00,2
2,06-10-2022,Computer Lab,N001,1,1,Thursday,06-10-2022 11:00:00,06-10-2022 13:00:00,3
3,10-10-2022,Lecture,CC012,1,2,Monday,10-10-2022 14:00:00,10-10-2022 16:00:00,4
4,13-10-2022,Computer Lab,N001,1,2,Thursday,13-10-2022 09:00:00,13-10-2022 11:00:00,5
5,13-10-2022,Computer Lab,N001,1,2,Thursday,13-10-2022 11:00:00,13-10-2022 13:00:00,6
6,17-10-2022,Lecture,CC012,1,3,Monday,17-10-2022 14:00:00,17-10-2022 16:00:00,7
7,20-10-2022,Computer Lab,N001,1,3,Thursday,20-10-2022 09:00:00,20-10-2022 11:00:00,8
8,20-10-2022,Computer Lab,N001,1,3,Thursday,20-10-2022 11:00:00,20-10-2022 13:00:00,9
9,24-10-2022,Lecture,CC012,1,4,Monday,24-10-2022 14:00:00,24-10-2022 16:00:00,10


#### Stage 4: Cleaning dfCOA11 and dfCOA122 i.e., dfCOA111Clean and dfCOA122Clean

    4.a Create a copy of the original dataframes.

In [171]:
dfCOA111Clean = dfCOA111.copy()
dfCOA122Clean = dfCOA122.copy()

    4.b Rename the columns of copied data frames with a unique session id/number.

In [172]:
# dfCOA111Clean.set_axis(['SID', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14'], axis = 1, inplace = True)
# dfCOA122Clean.set_axis(['SID', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14',
# 'S15', 'S16', 'S17', 'S18', 'S19'], axis = 1, inplace = True)

dfCOA111Clean.set_axis(['SID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], axis = 1, inplace = True)
dfCOA122Clean.set_axis(['SID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], axis = 1, inplace = True)

    4.c Replace the following: 
        - 'Ex' with a null value
        - 'GPS' with True (boolean)
        - 'X' with False (boolean)

In [173]:
# COA111
dfCOA111Clean.replace('Ex', None, inplace = True)
dfCOA111Clean.replace('GPS', True, inplace = True)
dfCOA111Clean.replace('X', False, inplace = True)

# COA122
dfCOA122Clean.replace('Ex', None, inplace = True)
dfCOA122Clean.replace('GPS', True, inplace = True)
dfCOA122Clean.replace('X', False, inplace = True)

    4.d Remove rows and columns which comprise of just NULL values.

In [174]:
 # drops rows where all values (aside from the student id column) are NULL
dfCOA111Clean.dropna(axis = 0, how = 'all', subset = dfCOA111Clean.columns[1:len(dfCOA111Clean.columns)], inplace = True)
dfCOA111Clean.dropna(axis = 1, how = 'all', inplace = True) # drops columns where all values are NULL

dfCOA122Clean.dropna(axis = 0, how = 'all', subset = dfCOA122Clean.columns[1:len(dfCOA122Clean.columns)], inplace = True)
dfCOA122Clean.dropna(axis = 1, how = 'all', inplace = True)

    4.d* Transpose the tables to enable the module attendance and sessions tables to be joined by session number

In [179]:
dfCOA111Transpose = dfCOA111Clean.transpose()
dfCOA111Transpose = dfCOA111Clean.transpose()
dfCOA111Transpose.drop(['SID'], axis = 0, inplace = True)
dfCOA111Transpose['Session_Number'] = dfCOA111Transpose.index
dfCOA111Transpose = dfCOA111Transpose.add_prefix('S')
dfCOA111Transpose.rename (
    columns = {
        'SSession_Number':'Session_Number'
    },
    inplace = True
)
dfCOA111Transpose

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S227,S228,S229,S230,S231,S232,S233,S234,S235,Session_Number
1,True,True,True,NaN,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,False,1
2,True,True,True,NaN,False,True,False,True,False,True,...,True,True,True,False,False,False,True,True,False,2
3,False,True,True,True,False,True,False,True,False,True,...,True,False,True,True,True,True,True,True,False,3
5,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,5
6,True,True,False,False,False,True,False,True,True,True,...,True,False,True,False,False,True,False,True,False,6
8,True,True,True,True,True,True,True,True,True,False,...,True,False,True,True,True,True,True,True,False,8
9,True,True,False,False,False,True,False,True,False,False,...,True,True,True,True,True,True,True,True,NaN,9
11,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,NaN,11
12,False,True,False,False,False,True,True,True,False,True,...,True,True,True,True,True,True,True,True,NaN,12
14,False,True,False,True,True,True,True,False,True,True,...,True,True,True,True,True,False,True,True,NaN,14


In [176]:
dfCOA122Transpose = dfCOA122Clean.transpose()
dfCOA122Transpose.drop(['SID'], axis = 0, inplace = True)
dfCOA122Transpose['Session_Number'] = dfCOA122Transpose.index
dfCOA122Transpose = dfCOA122Transpose.add_prefix('S')
dfCOA122Transpose.rename (
    columns = {
        'SSession_Number':'Session_Number'
    },
    inplace = True
)
dfCOA122Transpose

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S227,S228,S229,S230,S231,S232,S233,S234,S235,Session_Number
1,True,True,True,NaN,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,False,1
2,True,True,True,NaN,True,NaN,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,False,True,NaN,True,NaN,2
3,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,True,...,True,True,False,False,NaN,NaN,True,NaN,False,3
4,True,True,True,NaN,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,4
5,False,True,True,False,False,NaN,NaN,True,False,NaN,...,NaN,NaN,NaN,NaN,True,True,NaN,True,NaN,5
6,NaN,NaN,NaN,False,NaN,True,False,NaN,NaN,True,...,True,True,True,True,NaN,NaN,True,NaN,NaN,6
7,False,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,7
8,False,True,True,NaN,False,NaN,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,True,False,NaN,True,NaN,8
9,NaN,NaN,NaN,True,NaN,True,False,NaN,NaN,True,...,True,False,True,True,NaN,NaN,True,NaN,NaN,9
10,True,True,True,False,False,True,True,True,True,True,...,True,False,True,True,True,False,True,True,False,10


    4.e Create a database in SQlite and save it as CWDatabase.db.
    Then create tables in sql and store the dataframes* into these tables.
    (*dfCOA111Sessions, dfCOA111Transpose, dfCOA122Sessions and dfCOA122Transpose)

In [181]:
try:
    conn = sqlite3.connect('CWDatabase.db') # connect to the empty database
    cursor = conn.cursor()

    ################## COA111 ##################

    # insert tables dfCOA111 and dfCOA111Sessions into database
    dfCOA111Transpose.to_sql('COA111', conn, if_exists = 'replace', index = False)
    dfCOA111Sessions.to_sql('COA111Sessions', conn, if_exists = 'replace', index = False)

    # check data type of tables 
    cursor.execute('PRAGMA table_info (COA111)')
    print('------- Original COA111: Column Name, Data Type -------')
    for result in cursor:
        print(result[1:3])

    cursor.execute('PRAGMA table_info (COA111Sessions)')
    print('------- Original COA111Sessions: Column Name, Data Type -------')
    for result in cursor:
        print(result[1:3])

    # correct data types of dfCOA111Sessions
    cursor.execute('PRAGMA foreign_keys = off;')
    cursor.execute('BEGIN TRANSACTION;')
    cursor.execute('ALTER TABLE COA111Sessions RENAME TO COA111Sessions_Old;') # makes a copy of the table
    cursor.execute(

        '''CREATE TABLE COA111Sessions (
            Date DATE,
            Lecture_Type VARCHAR, 
            Room_ID VARCHAR, 
            Semester INTEGER, 
            Week INTEGER, 
            Day VARCHAR, 
            Start_Time TIMESTAMP, 
            End_Time TIMESTAMP, 
            Session_Number INTEGER);
        '''
    )
    cursor.execute(
        '''
        INSERT INTO COA111Sessions (
            Date, 
            Lecture_Type, 
            Room_ID, 
            Semester, 
            Week, 
            Day, 
            Start_Time, 
            End_Time, 
            Session_Number) 
        SELECT Date, Lecture_Type, Room_ID, Semester, Week, Day, Start_Time, End_Time, Session_Number 
        FROM COA111Sessions_Old;
        '''
    )
    cursor.execute('DROP TABLE COA111Sessions_Old;') # drop table to 'free-up' resources
    cursor.execute('COMMIT;')
    cursor.execute('PRAGMA foreign_keys = on;')
    
    # check if data types have been corrected and print the result
    cursor.execute('PRAGMA table_info (COA111Sessions)') # returns column name and its data type
    print('------- Corrected COA111Sessions: Column Name, Data Type -------')
    for result in cursor:
        print(result[1:3])
    
    ############################################


    ################## COA122 ##################

    # insert tables dfCOA122 and dfCOA122Sessions into database
    dfCOA122Transpose.to_sql('COA122', conn, if_exists = 'replace', index = False)
    dfCOA122Sessions.to_sql('COA122Sessions', conn, if_exists = 'replace', index = False)

        # check data type of tables 
    cursor.execute('PRAGMA table_info (COA122)')
    print('------- Original COA122: Column Name, Data Type -------')
    for result in cursor:
        print(result[1:3])

    cursor.execute('PRAGMA table_info (COA122Sessions)')
    print('------- Original COA122Sessions: Column Name, Data Type -------')
    for result in cursor:
        print(result[1:3])

    # correct data types of dfCOA122Sessions
    cursor.execute('PRAGMA foreign_keys = off;')
    cursor.execute('BEGIN TRANSACTION;')
    cursor.execute('ALTER TABLE COA122Sessions RENAME TO COA122Sessions_Old;') # makes a copy of the table
    cursor.execute(

        '''CREATE TABLE COA122Sessions (
            Date DATE,
            Lecture_Type VARCHAR, 
            Room_ID VARCHAR, 
            Semester INTEGER, 
            Week INTEGER, 
            Day VARCHAR, 
            Start_Time TIMESTAMP, 
            End_Time TIMESTAMP, 
            Session_Number INTEGER);
        '''
    )
    cursor.execute(
        '''
        INSERT INTO COA122Sessions (
            Date, 
            Lecture_Type, 
            Room_ID, 
            Semester, 
            Week, 
            Day, 
            Start_Time, 
            End_Time, 
            Session_Number) 
        SELECT Date, Lecture_Type, Room_ID, Semester, Week, Day, Start_Time, End_Time, Session_Number 
        FROM COA122Sessions_Old;
        '''
    )
    cursor.execute('DROP TABLE COA122Sessions_Old;') 
    cursor.execute('COMMIT;')
    cursor.execute('PRAGMA foreign_keys = on;')
    cursor.execute('PRAGMA table_info (COA111Sessions)')

    print('------- Corrected COA122Sessions: Column Name, Data Type -------')
    for result in cursor:
        print(result[1:3])
    
    ############################################

    conn.close() # close connection to database

except sqlite3.Error as error:
    print('Failed to insert table into the database', error)

------- Original COA111: Column Name, Data Type -------
('S0', 'INTEGER')
('S1', 'INTEGER')
('S2', 'INTEGER')
('S3', 'INTEGER')
('S4', 'INTEGER')
('S5', 'INTEGER')
('S6', 'INTEGER')
('S7', 'INTEGER')
('S8', 'INTEGER')
('S9', 'INTEGER')
('S10', 'INTEGER')
('S11', 'INTEGER')
('S12', 'INTEGER')
('S13', 'INTEGER')
('S15', 'INTEGER')
('S16', 'INTEGER')
('S18', 'INTEGER')
('S19', 'INTEGER')
('S20', 'INTEGER')
('S21', 'INTEGER')
('S22', 'INTEGER')
('S23', 'INTEGER')
('S24', 'INTEGER')
('S25', 'INTEGER')
('S26', 'INTEGER')
('S27', 'INTEGER')
('S28', 'INTEGER')
('S30', 'INTEGER')
('S31', 'INTEGER')
('S32', 'INTEGER')
('S34', 'INTEGER')
('S35', 'INTEGER')
('S36', 'INTEGER')
('S37', 'INTEGER')
('S38', 'INTEGER')
('S39', 'INTEGER')
('S40', 'INTEGER')
('S41', 'INTEGER')
('S42', 'INTEGER')
('S43', 'INTEGER')
('S44', 'INTEGER')
('S45', 'INTEGER')
('S47', 'INTEGER')
('S48', 'INTEGER')
('S49', 'INTEGER')
('S51', 'INTEGER')
('S52', 'INTEGER')
('S53', 'INTEGER')
('S54', 'INTEGER')
('S55', 'INTEGER')
('S5

# Testing 

In [178]:
# Test case (1)